## Importar bibliotecas necessárias

In [ ]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo dos gráficos
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## Configurar URL da API

⚠️ **Importante**: Certifique-se de que a API está rodando antes de executar este notebook!

Para iniciar a API, execute em outro terminal:
```bash
cd samples-3-ai
uvicorn main:app --reload
```

In [ ]:
# URL base da API
API_URL = "http://localhost:8000"

print(f"API URL configurada: {API_URL}")
print(f"Documentação disponível em: {API_URL}/docs")

## 1. Verificar Status da API

Vamos verificar se a API está funcionando e se o modelo foi carregado corretamente.

In [ ]:
# Testar endpoint raiz
response = requests.get(f"{API_URL}/")

if response.status_code == 200:
    print("✅ API está funcionando!")
    print("\nInformações da API:")
    print(json.dumps(response.json(), indent=2, ensure_ascii=False))
else:
    print(f"❌ Erro ao conectar com a API: {response.status_code}")

In [ ]:
# Verificar saúde da API
response = requests.get(f"{API_URL}/health")

if response.status_code == 200:
    health = response.json()
    print("Status da API:")
    print(json.dumps(health, indent=2, ensure_ascii=False))
    
    if health['model_loaded']:
        print("\n✅ Modelo carregado e pronto para fazer predições!")
    else:
        print("\n⚠️ Modelo não foi carregado. Verifique se o arquivo existe em outputs/")
else:
    print(f"❌ Erro ao verificar saúde da API: {response.status_code}")

## 2. Listar Features do Modelo

Vamos ver quais são as 30 características que o modelo espera.

In [ ]:
# Obter lista de features
response = requests.get(f"{API_URL}/features")

if response.status_code == 200:
    features_info = response.json()
    print(f"Número de features: {features_info['num_features']}")
    print(f"\nDescrição: {features_info['description']}")
    print(f"\n{features_info['note']}")
    print("\nFeatures esperadas:")
    
    # Criar DataFrame para melhor visualização
    df_features = pd.DataFrame({
        'Índice': range(len(features_info['feature_names'])),
        'Nome da Feature': features_info['feature_names']
    })
    
    display(df_features)
else:
    print(f"❌ Erro ao obter features: {response.status_code}")

## 3. Casos de Teste para Predição

Vamos criar três casos de teste com as **10 features** que o modelo espera:
1. `radius` - Raio médio
2. `texture` - Textura
3. `perimeter` - Perímetro
4. `area` - Área
5. `smoothness` - Suavidade
6. `compactness` - Compacidade
7. `concavity` - Concavidade
8. `concave points` - Pontos côncavos
9. `symmetry` - Simetria
10. `fractal dimension` - Dimensão fractal

- **Caso 1**: Características típicas de tumor benigno
- **Caso 2**: Características típicas de tumor maligno
- **Caso 3**: Características intermediárias

In [ ]:
# Caso 1: Tumor provavelmente BENIGNO (valores menores)
# Features: radius, texture, perimeter, area, smoothness, compactness, concavity, concave points, symmetry, fractal dimension
caso1_benigno = {
    "features": [
        11.0,   # radius
        16.0,   # texture
        70.0,   # perimeter
        380.0,  # area
        0.08,   # smoothness
        0.05,   # compactness
        0.02,   # concavity
        0.01,   # concave points
        0.17,   # symmetry
        0.06    # fractal dimension
    ]
}

# Caso 2: Tumor provavelmente MALIGNO (valores maiores)
caso2_maligno = {
    "features": [
        20.0,   # radius
        25.0,   # texture
        130.0,  # perimeter
        1200.0, # area
        0.12,   # smoothness
        0.25,   # compactness
        0.30,   # concavity
        0.15,   # concave points
        0.25,   # symmetry
        0.08    # fractal dimension
    ]

}

# Caso 3: Tumor com características INTERMEDIÁRIASprint(f"   - Caso 1: {len(caso1_benigno['features'])} features (Provável Benigno)")
caso3_intermediario = {
    "features": [
        15.0,   # radius
        20.0,   # texture
        95.0,   # perimeter
        700.0,  # area
        0.10,   # smoothness
        0.15,   # compactness
        0.15,   # concavity
        0.08,   # concave points
        0.20,   # symmetry
        0.07    # fractal dimension
    ]
}

## 4. Fazer Predições

Agora vamos enviar os casos para a API e obter as predições.

In [ ]:
def fazer_predicao(caso, nome_caso):
    """Função auxiliar para fazer predição e exibir resultado"""
    response = requests.post(f"{API_URL}/predict", json=caso)
    
    if response.status_code == 200:
        resultado = response.json()
        print(f"\n{'='*60}")
        print(f"📊 {nome_caso}")
        print(f"{'='*60}")
        print(f"\n🔬 Diagnóstico: {resultado['prediction_label']} ({resultado['prediction']})")
        print(f"\n📈 Confiança: {resultado['confidence']:.2%}")
        print(f"\n📊 Probabilidades:")
        for classe, prob in resultado['probability'].items():
            label = "Benigno" if classe == "0" else "Maligno"
            print(f"   {label}: {prob:.4f} ({prob*100:.2f}%)")
        
        # Análise da confiança
        if resultado['confidence'] > 0.9:
            print(f"\n✅ Alta confiança na predição")
        elif resultado['confidence'] > 0.7:
            print(f"\n⚠️  Confiança moderada - recomenda-se análise adicional")
        else:
            print(f"\n⚠️  Baixa confiança - resultado inconclusivo")
        
        return resultado
    else:
        print(f"❌ Erro ao fazer predição: {response.status_code}")
        print(f"   Detalhes: {response.text}")
        return None

# Fazer predições para todos os casos
resultado1 = fazer_predicao(caso1_benigno, "Caso 1 - Provável Benigno")
resultado2 = fazer_predicao(caso2_maligno, "Caso 2 - Provável Maligno")
resultado3 = fazer_predicao(caso3_intermediario, "Caso 3 - Intermediário")

## 5. Visualizar Resultados

Vamos criar visualizações para comparar os resultados das predições.

In [ ]:
# Consolidar resultados
resultados = [
    ("Caso 1\n(Benigno)", resultado1),
    ("Caso 2\n(Maligno)", resultado2),
    ("Caso 3\n(Intermediário)", resultado3)
]

# Criar DataFrame para visualização
df_resultados = pd.DataFrame([
    {
        'Caso': nome,
        'Diagnóstico': r['prediction_label'],
        'Confiança': r['confidence'],
        'Prob. Benigno': r['probability'].get('0', 0),
        'Prob. Maligno': r['probability'].get('1', 0)
    }
    for nome, r in resultados if r is not None
])

print("\nResumo dos Resultados:")
display(df_resultados)

In [ ]:
# Visualização 1: Gráfico de barras com probabilidades
fig, ax = plt.subplots(figsize=(12, 6))

x = range(len(df_resultados))
width = 0.35

bars1 = ax.bar([i - width/2 for i in x], df_resultados['Prob. Benigno'], 
               width, label='Benigno', color='skyblue', alpha=0.8)
bars2 = ax.bar([i + width/2 for i in x], df_resultados['Prob. Maligno'], 
               width, label='Maligno', color='salmon', alpha=0.8)

ax.set_xlabel('Casos', fontsize=12)
ax.set_ylabel('Probabilidade', fontsize=12)
ax.set_title('Probabilidades de Diagnóstico por Caso', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(df_resultados['Caso'])
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)

# Adicionar valores nas barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1%}',
                ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Visualização 2: Gráfico de confiança
fig, ax = plt.subplots(figsize=(10, 6))

colors = ['green' if c > 0.9 else 'orange' if c > 0.7 else 'red' 
          for c in df_resultados['Confiança']]

bars = ax.barh(df_resultados['Caso'], df_resultados['Confiança'], 
               color=colors, alpha=0.7)

ax.set_xlabel('Nível de Confiança', fontsize=12)
ax.set_title('Confiança da Predição por Caso', fontsize=14, fontweight='bold')
ax.set_xlim(0, 1)
ax.grid(axis='x', alpha=0.3)

# Adicionar linhas de referência
ax.axvline(x=0.7, color='orange', linestyle='--', alpha=0.5, label='Confiança Moderada')
ax.axvline(x=0.9, color='green', linestyle='--', alpha=0.5, label='Alta Confiança')

# Adicionar valores nas barras
for i, (bar, conf) in enumerate(zip(bars, df_resultados['Confiança'])):
    ax.text(conf + 0.02, bar.get_y() + bar.get_height()/2, 
            f'{conf:.1%}',
            ha='left', va='center', fontsize=11, fontweight='bold')

ax.legend(loc='lower right')
plt.tight_layout()
plt.show()

## 6. Teste Interativo (Opcional)

Você pode criar seus próprios casos de teste modificando os valores abaixo.

In [ ]:
# Crie seu próprio caso de teste modificando os valores abaixo
meu_caso = {
    "features": [
        14.0,   # radius
        19.0,   # texture
        90.0,   # perimeter
        600.0,  # area
        0.095,  # smoothness
        0.10,   # compactness
        0.12,   # concavity
        0.05,   # concave points
        0.18,   # symmetry
        0.065   # fractal dimension
    ]
}

# Fazer predição
meu_resultado = fazer_predicao(meu_caso, "Meu Caso Personalizado")